<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Wed May 19 10:32:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 67.9MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 8.6MB/s 
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 901kB 46.4MB/s 


## Import Packages

In [5]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from random import randint

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(20)

In [6]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.4MB/s 


## Load Model and Tokenizer




 

In [7]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
# tokenizer = AutoTokenizer.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

# model = AutoModelForQuestionAnswering.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("wptoux/albert-chinese-large-qa")

model = AutoModelForQuestionAnswering.from_pretrained("wptoux/albert-chinese-large-qa")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [8]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [9]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 128

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            if answer_end_token - answer_start_token <= 0:
                paragraph_start = 0
            else:
                pos = randint(answer_start_token, answer_end_token)
                paragraph_start = max(0, min(pos - int(self.max_paragraph_len * (pos - answer_start_token) / (answer_end_token - answer_start_token)), len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
# temp_set = ConcatDataset([train_set, dev_set])
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [13]:
def evaluate(data, output, index):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            if answer.find('[UNK]') != -1:
                countUNK = answer.count('[UNK]')
                flag = 0
                print(f'ID {index} Before : {answer}')
                while answer.find('[UNK]') != -1:
                    window_len = end_index - start_index + 10
                    start = 0
                    end = window_len
                    if answer.find('[UNK]') > 1:
                        target1 = answer[answer.find('[UNK]') - 2]
                    else:
                        target1 = -1
                    if len(answer) - (answer.find('[UNK]') + len('[UNK]')) > 0:
                        target2 = answer[answer.find('[UNK]') + len('[UNK]') + 1]
                    else:
                        target2 = -1
                    count = 0
                    for i in range(len(test_paragraphs[test_questions[index]['paragraph_id']])):
                        if end > len(test_paragraphs[test_questions[index]['paragraph_id']]):
                            final = len(test_paragraphs[test_questions[index]['paragraph_id']])
                        else:
                            final = end + i + 1
                        para = test_paragraphs[test_questions[index]['paragraph_id']][start+i: final]
                        if target1 != -1 and target2 != -1:
                            if para.find(target1) != -1 and para.find(target2) != -1:
                                target = para[para.find(target1) + 1: para.find(target2)]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 != -1 and target2 == -1:
                            if para.find(target1) != -1 and para.find(target1) != len(para) - 1:
                                target = para[para.find(target1) + 1]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 == -1 and target2 != -1:
                            if para.find(target2) != -1 and para.find(target2) != 0:
                                target = para[para.find(target2) - 1]
                                print(f'Target After: {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        else:
                            answer = answer.replace('[UNK]', para[answer.find('[UNK]')])
                            flag = 1
                            break
                        if count >= countUNK:
                            break
                        if i == len(test_paragraphs[test_questions[index]['paragraph_id']]) - 1:
                            flag = 1
                    if flag == 1:
                      break
                        
    answer = answer.replace('[SEP]','')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [14]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

accum_iter = 4 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        with torch.set_grad_enabled(True):
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            output.loss = output.loss / accum_iter

            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
            
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            scheduler.step()

            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 2.766, acc = 0.282
Epoch 1 | Step 200 | loss = 1.996, acc = 0.410
Epoch 1 | Step 300 | loss = 1.491, acc = 0.532
Epoch 1 | Step 400 | loss = 1.175, acc = 0.579
Epoch 1 | Step 500 | loss = 0.976, acc = 0.641
Epoch 1 | Step 600 | loss = 0.926, acc = 0.673
Epoch 1 | Step 700 | loss = 0.774, acc = 0.716
Epoch 1 | Step 800 | loss = 0.631, acc = 0.726
Epoch 1 | Step 900 | loss = 0.764, acc = 0.686
Epoch 1 | Step 1000 | loss = 0.667, acc = 0.738
Epoch 1 | Step 1100 | loss = 0.650, acc = 0.720
Epoch 1 | Step 1200 | loss = 0.669, acc = 0.752
Epoch 1 | Step 1300 | loss = 0.676, acc = 0.738
Epoch 1 | Step 1400 | loss = 0.724, acc = 0.727
Epoch 1 | Step 1500 | loss = 0.610, acc = 0.745
Epoch 1 | Step 1600 | loss = 0.614, acc = 0.750
Epoch 1 | Step 1700 | loss = 0.655, acc = 0.744
Epoch 1 | Step 1800 | loss = 0.587, acc = 0.765
Epoch 1 | Step 1900 | loss = 0.569, acc = 0.759
Epoch 1 | Step 2000 | loss = 0.615, acc = 0.761
Epoch 1 | Step 2100 | loss = 0.505, acc = 0.785
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 58 Before : 敵 對 同 盟 中 。 為 此 科 穆 寧 皇 帝 們 在 通 過 外 交 軍 事 雙 管 齊 下 暫 時 安 穩 住 佩 徹 涅 格 人 和 塞 爾 維 亞 人 後 ， 決 定 將 反 帝 國 同 盟 的 核 心 ， 始 終 敵 視 帝 國 的 諾 曼 人 趕 出 西 西 里 ， 為 此 引 發 了 與 諾 曼 人 的 海 上 衝 突 ， 但 最 終 消 滅 諾 曼 人 王 國 ， 重 建 帝 國 在 南 義 大 利 統 治 的 戰 略 意 圖 仍 以 失 敗 告 終 ， 只 能 迫 使 諾 曼 人 在 相 當 一 段 長 時 間 無 法 進 犯 帝 國 。 此 外 東 羅 馬 帝 國 還 對 匈 牙 利 王 國 發 動 了 兩 次 戰 爭 ， 雖 然 結 果 令 匈 牙 利 國 王 臣 服 ， 並 使 帝 國 獲 得 對 克 羅 埃 西 亞 地 區 的 直 轄 權 ， 但 耗 費 了 大 量 的 軍 力 ， 以 致 帝 國 近 東 的 小 亞 細 亞 國 防 防 禦 再 度 出 現 空 虛 。 1176 年 ， 在 相 繼 逼 迫 諾 曼 人 和 談 、 羈 [UNK] 匈 牙 利 、 取 締 威 尼 斯 貿 易 特 權 ， 並 通 過 資 助 倫 巴 第 同 盟 拖 住 紅 鬍 子 腓 特 烈 的 帝 國 統 一 後 ， 自 覺 西 方 邊 疆 已 經 穩 固 的 曼 努 埃 爾 一 [SEP]
ID 67 Before : 祭 天 、 祈 穀 和 祈 雨 的 場 所 。 是 現 存 中 國 古 代 規 模 最 大 、 倫 理 等 級 最 高 的 祭 祀 建 築 群 。 1961 年 ， 天 壇 被 中 華 人 民 共 和 國 國 務 院 公 布 為 第 一 批 全 國 重 點 文 物 保 護 單 位 之 一 。 1998 年 ， 「 北 京 皇 家 祭 壇 [UNK] 天 壇 」 被 列 為 世 界 文 化 遺 產 。 北 京 天 壇 最 初 為 明 永 樂 十 八 年 仿 南 京 城 形 制 而 建 的 天 地 壇 ， 嘉 靖 九 年 實 行 四 郊 分 祀 制 度 後 ， 在 北 郊 覓 地 另 

Epoch 2 | Step 100 | loss = 0.419, acc = 0.844
Epoch 2 | Step 200 | loss = 0.449, acc = 0.790
Epoch 2 | Step 300 | loss = 0.444, acc = 0.803
Epoch 2 | Step 400 | loss = 0.408, acc = 0.822
Epoch 2 | Step 500 | loss = 0.454, acc = 0.806
Epoch 2 | Step 600 | loss = 0.434, acc = 0.809
Epoch 2 | Step 700 | loss = 0.456, acc = 0.808
Epoch 2 | Step 800 | loss = 0.515, acc = 0.795
Epoch 2 | Step 900 | loss = 0.475, acc = 0.803
Epoch 2 | Step 1000 | loss = 0.453, acc = 0.795
Epoch 2 | Step 1100 | loss = 0.469, acc = 0.786
Epoch 2 | Step 1200 | loss = 0.449, acc = 0.797
Epoch 2 | Step 1300 | loss = 0.434, acc = 0.800
Epoch 2 | Step 1400 | loss = 0.389, acc = 0.829
Epoch 2 | Step 1500 | loss = 0.405, acc = 0.829
Epoch 2 | Step 1600 | loss = 0.414, acc = 0.825
Epoch 2 | Step 1700 | loss = 0.496, acc = 0.801
Epoch 2 | Step 1800 | loss = 0.454, acc = 0.797
Epoch 2 | Step 1900 | loss = 0.442, acc = 0.810
Epoch 2 | Step 2000 | loss = 0.386, acc = 0.839
Epoch 2 | Step 2100 | loss = 0.405, acc = 0.829
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 39 Before : 新 加 坡 在 近 十 五 人 均 [UNK] 都 比 香 港
ID 70 Before : 2002 年 8 月 15 日 正 式 運 營 並 成 為 國 內 率 先 成 功 運 營 的 國 產 網 路 遊 戲 ， 2013 年 9 月 5 日 ， 大 話 西 遊 [UNK] 全 新 升 級 為 新 大 話 西 遊 2 。 目 前 大 話 西 遊 2 品 牌 旗 下 產 品 註 冊 用 戶 近 2 億 ， 單 品 最 高 [SEP]
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 716 Before : 姚 [UNK]
ID 771 Before : 隋 文 帝 在 位 時 ， 為 防 範 突 厥 ， 多 次 於 冬 季 徵 發 丁 壯 修 築 長 城 。 公 元 585 年 ， 隋 文 帝 使 司 農 少 卿 崔 仲 方 發 丁 三 萬 於 朔 方 靈 武 築 長 城 ， 東 距 河 ， 西 至 綏 州 ， 綿 垣 七 百 里 ， 以 遏 胡 寇 。 隋 [UNK] 帝
ID 854 Before : [UNK]
ID 869 Before : 翁 同 [UNK] 起 草 《 明 定 國 是 詔 》 ， 送 交 慈 禧
ID 883 Before : [UNK] 水 之 戰
ID 937 Before : 長 [UNK] 米
ID 937 Before : 長 [UNK] 米
ID 950 Before : 長 社 縣 等 皆 屬 [UNK] 川 郡 。 許 昌 為 魏 五 都 之 一 。 魏 黃 初 二 年 ， 文 帝 曹 丕 以 漢 亡 於 許 ， 魏 基 昌 於 許 ， 改 許 縣 為 許 昌 縣 。 元 代 許 昌 仍 為 州 ， 長 社 、 長 葛 、 襄 城 縣 屬 之 。 鄢 陵 屬 開 封 府 。 明 洪 武 元 年 ， 廢 長 社 縣 ， 併 入 許 州 ， 領 4 縣 ， 長 葛 、 襄 城
ID 1118 Before : 金 [UNK]

Epoch 3 | Step 100 | loss = 0.330, acc = 0.816
Epoch 3 | Step 200 | loss = 0.334, acc = 0.855
Epoch 3 | Step 300 | loss = 0.391, acc = 0.835
Epoch 3 | Step 400 | loss = 0.395, acc = 0.830
Epoch 3 | Step 500 | loss = 0.356, acc = 0.842
Epoch 3 | Step 600 | loss = 0.338, acc = 0.852
Epoch 3 | Step 700 | loss = 0.362, acc = 0.851
Epoch 3 | Step 800 | loss = 0.362, acc = 0.844
Epoch 3 | Step 900 | loss = 0.393, acc = 0.830
Epoch 3 | Step 1000 | loss = 0.341, acc = 0.855
Epoch 3 | Step 1100 | loss = 0.279, acc = 0.871
Epoch 3 | Step 1200 | loss = 0.353, acc = 0.849
Epoch 3 | Step 1300 | loss = 0.343, acc = 0.844
Epoch 3 | Step 1400 | loss = 0.392, acc = 0.832
Epoch 3 | Step 1500 | loss = 0.304, acc = 0.859
Epoch 3 | Step 1600 | loss = 0.385, acc = 0.826
Epoch 3 | Step 1700 | loss = 0.395, acc = 0.830
Epoch 3 | Step 1800 | loss = 0.398, acc = 0.824
Epoch 3 | Step 1900 | loss = 0.407, acc = 0.842
Epoch 3 | Step 2000 | loss = 0.335, acc = 0.849
Epoch 3 | Step 2100 | loss = 0.324, acc = 0.852
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 39 Before : 新 加 坡 在 近 十 五 人 均 [UNK] 都 比 香 港
ID 58 Before : 敵 對 同 盟 中 。 為 此 科 穆 寧 皇 帝 們 在 通 過 外 交 軍 事 雙 管 齊 下 暫 時 安 穩 住 佩 徹 涅 格 人 和 塞 爾 維 亞 人 後 ， 決 定 將 反 帝 國 同 盟 的 核 心 ， 始 終 敵 視 帝 國 的 諾 曼 人 趕 出 西 西 里 ， 為 此 引 發 了 與 諾 曼 人 的 海 上 衝 突 ， 但 最 終 消 滅 諾 曼 人 王 國 ， 重 建 帝 國 在 南 義 大 利 統 治 的 戰 略 意 圖 仍 以 失 敗 告 終 ， 只 能 迫 使 諾 曼 人 在 相 當 一 段 長 時 間 無 法 進 犯 帝 國 。 此 外 東 羅 馬 帝 國 還 對 匈 牙 利 王 國 發 動 了 兩 次 戰 爭 ， 雖 然 結 果 令 匈 牙 利 國 王 臣 服 ， 並 使 帝 國 獲 得 對 克 羅 埃 西 亞 地 區 的 直 轄 權 ， 但 耗 費 了 大 量 的 軍 力 ， 以 致 帝 國 近 東 的 小 亞 細 亞 國 防 防 禦 再 度 出 現 空 虛 。 1176 年 ， 在 相 繼 逼 迫 諾 曼 人 和 談 、 羈 [UNK] 匈 牙 利 、 取 締 威 尼 斯 貿 易 特 權 ， 並 通 過 資 助 倫 巴 第 同 盟 拖 住 紅 鬍 子 腓 特 烈 的 帝 國 統 一 後 ， 自 覺 西 方 邊 疆 已 經 穩 固 的 曼 努 埃 爾 一 [SEP]
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 345 Before : 暴 力 鎮 壓 。 為 了 控 制 不 同 民 族 和 文 化 的 紅 軍 士 兵 ， 契 卡 操 控 特 別 懲 戒 旅 鎮 壓 反 共 人 士 、 逃 兵 和 國 家 的 敵 人 。 戰 時 實 用 主 義 允 許 招 募 前 沙 皇 時 期 軍 官 和 警 官 到 紅 軍 ， 由 [UNK] [UNK] 的 特 別 委 員 會 審 查 並 徵 召 ； 1920 年

In [15]:
test_paragraphs[test_questions[49]['paragraph_id']]

'部份香港政府機關在九龍灣設置總部，包括機電工程署總部大樓、香港輔助警察隊總部，郵政署亦於2013年在該處設立中央郵件中心。附近亦有九龍灣運動場、公園、體育館等公共設施。九龍灣國際展貿中心是香港主要的國際商品展覽會場之一，附設的辦公室亦可作不同產品陳列室之用。現時國際展貿中心仍繼續舉辦不同的國際展覽，亦會不時舉辦演唱會。2007年，自大型購物中心MegaBox開幕後，改變了九龍灣作為工廠、物流倉庫的印象。MegaBox商場高18層，佔地達110萬平方呎，是觀塘區最大型的購物商場，當中的大型商戶有宜家傢俬、AEON、IMAX影院等。區內的寫字樓自2006年起不斷增建，包括企業廣場三期、國際交易中心、Manhattan Place、傲騰廣場及億京中心等，成為東九龍新商業中心。2012年，香港首座零碳建築零碳天地落成，為香港政府宣布《起動九龍東》計劃後，成為九龍東首批標誌性項目之一。九龍灣也有一些住宅區，有兩大私人屋苑，分別是九龍灣站上蓋的德福花園及麗晶花園。'

In [16]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if result[0] == '《' and result[-1] != '》' and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result[0] != '《' and result[-1] == '》' and result.find('《') == -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')
        
    return result

## Testing

In [17]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, i))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


ID 26 Before : 拉 丁 文 [UNK]
Target : 中
ID 26 After  : 拉 丁 文 中
ID 49 Before : 大 型 購 物 中 心 [UNK] 開 幕
Target : MegaBox
ID 49 After  : 大 型 購 物 中 心 MegaBox 開 幕
ID 49 Before : 大 型 購 物 中 心 [UNK] 開 幕
Target : MegaBox
ID 49 After  : 大 型 購 物 中 心 MegaBox 開 幕
ID 84 Before : [UNK] [SEP]
ID 125 Before : 缺 乏 標 準 化 ， 它 也 可 以 寫 作 [UNK] 喃 或 [UNK] 喃 等 。 據 推 測 ， 喃 字 大 概 是 從 10 世 紀 越 南 脫 離 中 國 的 直 接 統 治 後 ， 才 開 始 發 展 出 來 。 早 期 的 喃 字 主 要 作 為 漢 字 的 輔 助 工 具 ， 用 在 記 錄 地 名 、 人 名 及 地 方 特 產 等 。 在 累 積 數 百 年 使 用 的 經 驗 後 ， 在 13 世 紀 才 有 一 些 字 喃 的 文 學 作 品 出 現 ， 而 於 16 至 18 世 紀 達 到 高 潮 。 譬 如 阮 攸 的 字 喃 著 作 《 金 雲 翹 傳 》 等 。 字 喃 的 使 用 者 大 致 為 平 民 、 落 魄 文 人 、 僧 侶 、 及 少 數 具 強 烈 民 族 意 識 的 精 英 。 概 括 來 說 ， 喃 字 主 要 用 在 紀 錄 民 間 口 傳 文 學 、 創 作 純 越 語 文 學 、 翻 譯 佛 [SEP]
Target : 
ID 125 After  : 缺 乏 標 準 化 ， 它 也 可 以 寫 作  喃 或  喃 等 。 據 推 測 ， 喃 字 大 概 是 從 10 世 紀 越 南 脫 離 中 國 的 直 接 統 治 後 ， 才 開 始 發 展 出 來 。 早 期 的 喃 字 主 要 作 為 漢 字 的 輔 助 工 具 ， 用 在 記 錄 地 名 、 人 名 及 地 方 特 產 等 。 在 累 積 數 百 年 使 用 的 經 驗 後 ， 在 13 世 紀 才 有 一 些 字 喃 的 文 學 作 品 出 現 ， 而 於 16 至 1

In [18]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>